In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import numpy as np

In [2]:
mbti= pd.read_csv('./final_project/mbti_changed.csv')

#### Binary prediction 

- Introvert: 1 - Extrovert: 0
- Intuition:1 - Sensing: 0
- Thinking: 1 - Sensing: 0
- Judging 1 - Perceiving: 0

In [3]:
def introvert(personality):
    
    if 'I' in personality:
        return 1
    else:
        return 0
mbti['introv_extrov']= mbti.apply(lambda x: introvert(x['type']), axis=1)

def intuition(personality):
    
    if 'N' in personality:
        return 1
    else:
        return 0
mbti['intuit_sensin']= mbti.apply(lambda x: intuition(x['type']), axis=1)

def thinking(personality):
    
    if 'T' in personality:
        return 1
    else:
        return 0
mbti['thinkin_feelin']= mbti.apply(lambda x: thinking(x['type']), axis=1)

def judging(personality):
    
    if 'J' in personality:
        return 1
    else:
        return 0
mbti['judg_percev']= mbti.apply(lambda x: judging(x['type']), axis=1)


In [4]:
mbti.head()

,type,posts,type_index,cleaned_text,introv_extrov,intuit_sensin,thinkin_feelin,judg_percev
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...,1,1,0,1
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...,0,1,1,0
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...,1,1,1,0
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...,1,1,1,1
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...,0,1,1,1


In [5]:
train,test= train_test_split(mbti)
train,val= train_test_split(train)

print(train.shape)
print(val.shape)
print(test.shape)

(4879, 8)
(1627, 8)
(2169, 8)


In [6]:
embed= hub.load("https://tfhub.dev/google/nnlm-en-dim50/2")
hub_layer=hub.KerasLayer(embed, input_shape=[], dtype=tf.string, trainable=True)

In [7]:
model=tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='softmax'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam', 
             loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=tf.metrics.BinaryAccuracy())

#### Introvertion vs Extrovertion

In [9]:
intro_extr= np.array(train['introv_extrov'])
intro_extr_val= np.array(val['introv_extrov'])

In [10]:
train_array= np.array(train['cleaned_text'])
val_array= np.array(val['cleaned_text'])

In [11]:
model.fit(train_array, intro_extr, validation_data=(val_array, intro_extr_val), verbose=1, epochs=10)

Epoch 1/10
153/153 [==============================] - 82s 536ms/step - loss: 0.6081 - binary_accuracy: 0.5380 - val_loss: 0.5646 - val_binary_accuracy: 0.7634
Epoch 2/10
153/153 [==============================] - 82s 535ms/step - loss: 0.5591 - binary_accuracy: 0.7682 - val_loss: 0.5512 - val_binary_accuracy: 0.7658
Epoch 3/10
153/153 [==============================] - 80s 521ms/step - loss: 0.5453 - binary_accuracy: 0.7709 - val_loss: 0.5461 - val_binary_accuracy: 0.7671
Epoch 4/10
153/153 [==============================] - 79s 518ms/step - loss: 0.5414 - binary_accuracy: 0.7701 - val_loss: 0.5445 - val_binary_accuracy: 0.7664
Epoch 5/10
153/153 [==============================] - 79s 519ms/step - loss: 0.5397 - binary_accuracy: 0.7677 - val_loss: 0.5141 - val_binary_accuracy: 0.7757
Epoch 6/10
153/153 [==============================] - 80s 522ms/step - loss: 0.4641 - binary_accuracy: 0.8586 - val_loss: 0.4337 - val_binary_accuracy: 0.8537
Epoch 7/10
153/153 [==========================

In [12]:
intro_extr= np.array(test['introv_extrov'])
test_array= np.array(test['cleaned_text'])

model.evaluate(test_array, intro_extr, verbose=1)

68/68 [==============================] - 0s 6ms/step - loss: 0.3922 - binary_accuracy: 0.8354


[0.39215466380119324, 0.8354080319404602]

#### Intuition vs Sensing

In [13]:
intuit_sens= np.array(train['intuit_sensin'])
intuit_sens_val= np.array(val['intuit_sensin'])
intuit_sens_test=np.array(test['intuit_sensin'])

In [14]:
model.fit(train_array, intuit_sens, validation_data=(val_array, intuit_sens_val), verbose=1, epochs=10)

Epoch 1/10
153/153 [==============================] - 83s 541ms/step - loss: 0.4184 - binary_accuracy: 0.8448 - val_loss: 0.4074 - val_binary_accuracy: 0.8611
Epoch 2/10
153/153 [==============================] - 82s 537ms/step - loss: 0.3957 - binary_accuracy: 0.8664 - val_loss: 0.4050 - val_binary_accuracy: 0.8611
Epoch 3/10
153/153 [==============================] - 81s 531ms/step - loss: 0.3942 - binary_accuracy: 0.8662 - val_loss: 0.4038 - val_binary_accuracy: 0.8611
Epoch 4/10
153/153 [==============================] - 81s 529ms/step - loss: 0.3931 - binary_accuracy: 0.8666 - val_loss: 0.4035 - val_binary_accuracy: 0.8605
Epoch 5/10
153/153 [==============================] - 83s 540ms/step - loss: 0.3927 - binary_accuracy: 0.8668 - val_loss: 0.4031 - val_binary_accuracy: 0.8611
Epoch 6/10
153/153 [==============================] - 83s 539ms/step - loss: 0.3925 - binary_accuracy: 0.8668 - val_loss: 0.4030 - val_binary_accuracy: 0.8611
Epoch 7/10
153/153 [==========================

In [15]:
model.evaluate(test_array, intuit_sens_test, verbose=1)

68/68 [==============================] - 0s 6ms/step - loss: 0.4194 - binary_accuracy: 0.8525


[0.4194318354129791, 0.8524665832519531]

#### Thinking vs Feeling

In [16]:
think_feel= np.array(train['thinkin_feelin'])
think_feel_val= np.array(val['thinkin_feelin'])
think_feel_test=np.array(test['thinkin_feelin'])

In [17]:
model.fit(train_array, think_feel, validation_data=(val_array, think_feel_val), verbose=1, epochs=10)

Epoch 1/10
153/153 [==============================] - 82s 538ms/step - loss: 0.9902 - binary_accuracy: 0.4526 - val_loss: 0.8570 - val_binary_accuracy: 0.4579
Epoch 2/10
153/153 [==============================] - 81s 527ms/step - loss: 0.8027 - binary_accuracy: 0.4534 - val_loss: 0.7558 - val_binary_accuracy: 0.4567
Epoch 3/10
153/153 [==============================] - 81s 529ms/step - loss: 0.7329 - binary_accuracy: 0.5327 - val_loss: 0.7157 - val_binary_accuracy: 0.5421
Epoch 4/10
153/153 [==============================] - 81s 528ms/step - loss: 0.7019 - binary_accuracy: 0.5468 - val_loss: 0.6941 - val_binary_accuracy: 0.5421
Epoch 5/10
153/153 [==============================] - 81s 529ms/step - loss: 0.6767 - binary_accuracy: 0.5468 - val_loss: 0.6841 - val_binary_accuracy: 0.5421
Epoch 6/10
153/153 [==============================] - 81s 531ms/step - loss: 0.6466 - binary_accuracy: 0.5468 - val_loss: 0.6298 - val_binary_accuracy: 0.5421
Epoch 7/10
153/153 [==========================

In [18]:
model.evaluate(test_array, think_feel_test, verbose=1)

68/68 [==============================] - 0s 6ms/step - loss: 0.4707 - binary_accuracy: 0.8087


[0.47073987126350403, 0.8086676001548767]

#### Judging vs Perceiving

In [19]:
judg_perceiv= np.array(train['judg_percev'])
judg_perceiv_val= np.array(val['judg_percev'])
judg_perceiv_test=np.array(test['judg_percev'])

In [20]:
model.fit(train_array, judg_perceiv, validation_data=(val_array, judg_perceiv_val), verbose=1, epochs=10)

Epoch 1/10
153/153 [==============================] - 81s 531ms/step - loss: 0.7521 - binary_accuracy: 0.5259 - val_loss: 0.7191 - val_binary_accuracy: 0.6023
Epoch 2/10
153/153 [==============================] - 82s 533ms/step - loss: 0.6484 - binary_accuracy: 0.5987 - val_loss: 0.6815 - val_binary_accuracy: 0.6023
Epoch 3/10
153/153 [==============================] - 81s 529ms/step - loss: 0.6031 - binary_accuracy: 0.5987 - val_loss: 0.6736 - val_binary_accuracy: 0.6023
Epoch 4/10
153/153 [==============================] - 81s 530ms/step - loss: 0.5765 - binary_accuracy: 0.5987 - val_loss: 0.6570 - val_binary_accuracy: 0.6023
Epoch 5/10
153/153 [==============================] - 81s 531ms/step - loss: 0.5631 - binary_accuracy: 0.5987 - val_loss: 0.6558 - val_binary_accuracy: 0.6023
Epoch 6/10
153/153 [==============================] - 81s 530ms/step - loss: 0.5527 - binary_accuracy: 0.5987 - val_loss: 0.6526 - val_binary_accuracy: 0.6023
Epoch 7/10
153/153 [==========================

In [21]:
model.evaluate(test_array, judg_perceiv_test, verbose=1)

68/68 [==============================] - 0s 6ms/step - loss: 0.6695 - binary_accuracy: 0.6178


[0.6695128083229065, 0.6177962422370911]